In [1]:
from src.utils import utils
from datasets import concatenate_datasets, interleave_datasets

/home/mourad/.conda/envs/mourad-econ-py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
now_ds = utils.load_hf_dataset(dataset='now')
proquest_ds = utils.load_hf_dataset(dataset='proquest')


In [40]:
now_ds['test']['template'][23]#.index('",\n    "inflation-direction": ')
# now_ds['test']['source']

'{\n"foreign": #,\n"contains-narrative": #,\n"inflation-narratives": {\n    "inflation-time": "#",\n    "narratives": #\n    }\n}'

In [32]:
def remove_distinct_fields(instance, version):
    template = instance['template'].split('#')
    try:
        if version == 'now':
            idx = template.index('",\n    "counter-narrative": ')
        else:
            idx = template.index('",\n    "inflation-direction": ')
    except:
        return instance
    
    template[idx+1] = '"' + template[idx+1] 
    instance['template'] = '#'.join(template[:idx] + template[idx+1:])    
    data = instance['data'].split('#')
    instance['data'] = "#".join(data[:idx] + data[idx+1:])
    return instance
    # print(instance)

In [33]:
splits = ['train', 'test', 'test_az', 'test_qz', 'test_mh']
for split in splits:
    now_ds[split] = now_ds[split].add_column('source', ['now']*len(now_ds[split]))
    proquest_ds[split] = proquest_ds[split].add_column('source', ['proquest']*len(proquest_ds[split]))
    now_ds[split] = now_ds[split].map(remove_distinct_fields, fn_kwargs={'version': 'now'})
    proquest_ds[split] = proquest_ds[split].map(remove_distinct_fields, fn_kwargs={'version': 'proquest'})
    now_ds[split] = concatenate_datasets([now_ds[split], proquest_ds[split]])
    now_ds[split] = now_ds[split].shuffle(seed=0)

Map:   0%|          | 0/1117 [00:00<?, ? examples/s]

Map: 100%|██████████| 488/488 [00:00<00:00, 14917.65 examples/s]


In [16]:
now_ds['test']['data']

['false#false',
 'false#false',
 'false#true#future#[{"cause": "monetary", "time": "future"}]',
 'false#true#present#[{"effect": "govt", "time": "present"}]',
 'false#true#future#[{"effect": "uncertain", "time": "present"}, {"effect": "savings", "time": "present"}]',
 'false#false',
 'false#false',
 'false#true#general#[{"cause": "other-cause", "time": "general"}]',
 'false#false',
 'false#false',
 'false#false',
 'false#true#present#[{"effect": "savings", "time": "present"}]',
 'false#true#present#[{"effect": "uncertain", "time": "present"}, {"effect": "savings", "time": "present"}]',
 'true#true#present#[{"effect": "trade", "time": "present"}, {"effect": "trade", "time": "present"}]',
 'false#false',
 'false#true#general#[{"cause": "fiscal", "time": "general"}]',
 'false#false',
 'false#false',
 'false#true#present#[{"effect": "other-effect", "time": "present"}]',
 'false#true#past#[{"cause": "supply", "time": "past"}, {"cause": "expect", "time": "past"}, {"cause": "international", "

In [41]:
now_ds.save_to_disk("/data/mourad/narratives/sft_data_now_and_proquest")

Saving the dataset (1/1 shards): 100%|██████████| 689/689 [00:00<00:00, 71046.21 examples/s]
